## Método: Descargar archivos Parquet y extraer imágenes

El dataset de Hugging Face está en formato Parquet. Como `pyarrow` no funciona en tu sistema, vamos a:
1. Descargar el archivo Parquet directamente
2. Usar una librería alternativa para leerlo
3. Extraer las imágenes

In [7]:
# Instalar fastparquet como alternativa a pyarrow
%pip install fastparquet

   ---------------------------------------- 0.0/673.3 kB ? eta -:--:--
   ---------------------------------------- 673.3/673.3 kB 4.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 15.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [12]:
from os import path


In [8]:
# Descargar el archivo Parquet del dataset
import pandas as pd

repo_id = "katanaml-org/invoices-donut-data-v1"
filename = "data/train-00000-of-00001-a5c51039eab2980a.parquet"

print(f"📥 Descargando archivo Parquet del dataset...")
print(f"   Repositorio: {repo_id}")
print(f"   Archivo: {filename}")

try:
    # Descargar el archivo usando huggingface_hub
    parquet_file = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        repo_type="dataset"
    )
    
    print(f"✅ Archivo descargado en: {parquet_file}")
    
    # Leer el archivo Parquet usando fastparquet
    df = pd.read_parquet(parquet_file, engine='fastparquet')
    
    print(f"✅ Datos cargados exitosamente")
    print(f"📊 Total de registros: {len(df)}")
    print(f"📋 Columnas: {list(df.columns)}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print(f"\\n💡 Intentando método alternativo...")

📥 Descargando archivo Parquet del dataset...
   Repositorio: katanaml-org/invoices-donut-data-v1
   Archivo: data/train-00000-of-00001-a5c51039eab2980a.parquet


data/train-00000-of-00001-a5c51039eab298(…):   0%|          | 0.00/167M [00:00<?, ?B/s]

✅ Archivo descargado en: C:\Users\Anuar\.cache\huggingface\hub\datasets--katanaml-org--invoices-donut-data-v1\snapshots\d2cde298e79c94fb05bc320999deb4b7889b0464\data\train-00000-of-00001-a5c51039eab2980a.parquet
✅ Datos cargados exitosamente
📊 Total de registros: 425
📋 Columnas: ['ground_truth', 'image.bytes', 'image.path']


In [20]:
# Extraer y guardar facturas
import json
import io
from pathlib import Path
from PIL import Image
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

# CORRECCIÓN: Usar Path() correctamente
facturas_dir = Path('../facturas_dataset')
facturas_dir.mkdir(parents=True, exist_ok=True)

num_samples = min(50, len(df))
print(f"\n💾 Guardando {num_samples} facturas de ejemplo...\n")

for i in range(num_samples):
    try:
        row = df.iloc[i]
        
        # La imagen está en la columna 'image.bytes' (no 'image')
        if 'image.bytes' in row:
            image_bytes = row['image.bytes']
            
            # Abrir la imagen desde bytes
            image = Image.open(io.BytesIO(image_bytes))
            
            # Guardar como PNG
            png_path = facturas_dir / f"factura_{i+1}.png"
            image.save(png_path, 'PNG')
            print(f"✅ Guardada imagen: {png_path.name}")
            
            # Convertir a PDF
            pdf_path = facturas_dir / f"factura_{i+1}.pdf"
            c = canvas.Canvas(str(pdf_path), pagesize=letter)
            
            # Calcular dimensiones
            img_width, img_height = image.size
            aspect = img_height / float(img_width)
            page_width, page_height = letter
            display_width = page_width - 40
            display_height = display_width * aspect
            
            if display_height > page_height - 40:
                display_height = page_height - 40
                display_width = display_height / aspect
            
            # Dibujar imagen
            c.drawImage(str(png_path), 20, page_height - display_height - 20,
                       width=display_width, height=display_height)
            c.save()
            print(f"✅ Creado PDF: {pdf_path.name}")
            
            # Guardar metadata
            metadata = {
                "numero": i + 1,
                "ground_truth": row.get('ground_truth', ''),
                "image_path": row.get('image.path', ''),
                "archivos": {
                    "imagen": f"factura_{i+1}.png",
                    "pdf": f"factura_{i+1}.pdf"
                }
            }
            
            metadata_path = facturas_dir / f"factura_{i+1}_metadata.json"
            with open(metadata_path, 'w', encoding='utf-8') as f:
                json.dump(metadata, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Metadata guardada\n")
            
    except Exception as e:
        print(f"❌ Error procesando factura {i+1}: {e}\n")
        continue

print(f"\n🎉 Proceso completado!")
print(f"📁 Facturas guardadas en: {facturas_dir.absolute()}")


💾 Guardando 50 facturas de ejemplo...

✅ Guardada imagen: factura_1.png
✅ Creado PDF: factura_1.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_2.png
✅ Creado PDF: factura_2.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_3.png
✅ Creado PDF: factura_3.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_4.png
✅ Creado PDF: factura_4.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_5.png
✅ Creado PDF: factura_5.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_6.png
✅ Creado PDF: factura_6.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_7.png
✅ Creado PDF: factura_7.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_8.png
✅ Creado PDF: factura_8.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_9.png
✅ Creado PDF: factura_9.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_10.png
✅ Creado PDF: factura_10.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_11.png
✅ Creado PDF: factura_11.pdf
✅ Metadata guardada

✅ Guardada imagen: factura_12.png
✅ Creado PDF: factur

## 🎯 Siguiente Paso

Ahora puedes usar las facturas descargadas en la interfaz web:

1. Ve a `http://localhost:3000/login`
2. Inicia sesión
3. Ve a `http://localhost:3000/extraccion`
4. Carga cualquiera de los archivos PDF de `facturas_dataset/`
5. Observa cómo OpenAI extrae los datos automáticamente
6. Compara con el ground truth en los archivos `_metadata.json`